In [306]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR

from sklearn.metrics import mean_squared_error
import optuna

In [219]:
from sklearn.impute import KNNImputer

In [220]:
train = pd.read_csv(r"D:\Data Science\Tea Story - Hackathon\train.csv")
test = pd.read_csv(r"D:\Data Science\Tea Story - Hackathon\test.csv")
sample = pd.read_csv(r"D:\Data Science\Tea Story - Hackathon\submission.csv")

In [221]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 544 entries, 0 to 543
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   WeekEnding_Date           544 non-null    object 
 1   Kolkata_Average_Price     544 non-null    object 
 2   Kolkata_Ref_Price         542 non-null    object 
 3   Bangalore_Average_Price   544 non-null    object 
 4   Bangalore_Ref_Price       542 non-null    object 
 5   Cochin_Average_Price      544 non-null    object 
 6   Cochin_Ref_Price          541 non-null    object 
 7   Darjeeling_Average_Price  544 non-null    object 
 8   Darjeeling_Ref_Price      542 non-null    object 
 9   Ernakulam_Average_Price   544 non-null    object 
 10  Ernakulam_Ref_Price       543 non-null    object 
 11  Siliguri_Average_Price    543 non-null    object 
 12  Siliguri_Ref_Price        543 non-null    object 
 13  Guwahati_Average_Price    544 non-null    object 
 14  Guwahati_R

In [222]:
pd.set_option('display.max_rows',None)

In [223]:
train.head()

,WeekEnding_Date,Kolkata_Average_Price,Kolkata_Ref_Price,Bangalore_Average_Price,Bangalore_Ref_Price,Cochin_Average_Price,Cochin_Ref_Price,Darjeeling_Average_Price,Darjeeling_Ref_Price,Ernakulam_Average_Price,Ernakulam_Ref_Price,Siliguri_Average_Price,Siliguri_Ref_Price,Guwahati_Average_Price,Guwahati_Ref_Price,Average
0,03/01/09,99.01,79.79,N.S.,76.19,84.02,70.07,81.66,57.83,68.94,51.67,70.74,53.88,65.55,46.75,69.700000
1,10/01/09,97.74,78.73,87.48,73.97,82.72,68.17,83.31,58.02,67.24,52.23,70.47,53.39,67.39,46.84,70.550000
2,17/01/09,95.95,71.01,87.66,71.01,80.58,67.16,82.25,57.49,69.64,52.48,71.66,53.18,69.51,48.04,69.830000
3,24/01/09,94.14,73.38,85.69,65.66,N.S.,65.57,80.87,54.59,N.S.,53.43,71.12,52.07,69.14,48.5,67.846667
4,31/01/09,91.45,70.39,N.S.,64.99,79.27,62.09,80.76,57.06,69.65,53.38,72.3,52.5,69.39,50.33,67.196923


In [224]:
test.head()

,WeekEnding_Date,Kolkata_Average_Price,Kolkata_Ref_Price,Bangalore_Average_Price,Bangalore_Ref_Price,Cochin_Average_Price,Cochin_Ref_Price,Darjeeling_Average_Price,Darjeeling_Ref_Price,Ernakulam_Average_Price,Ernakulam_Ref_Price,Siliguri_Average_Price,Siliguri_Ref_Price,Guwahati_Average_Price,Guwahati_Ref_Price,Average
0,15/06/19,NaN,165.87,154.08,160.82,NaN,NaN,116.45,112.82,91.82,NaN,102.30,NaN,83.07,65.34,NaN
1,22/06/19,171.35,173.12,NaN,NaN,NaN,NaN,NaN,NaN,90.68,76.34,NaN,NaN,81.34,NaN,NaN
2,29/06/19,NaN,NaN,156.14,NaN,NaN,150.39,NaN,NaN,NaN,NaN,96.84,NaN,NaN,NaN,NaN
3,06/07/19,NaN,NaN,158.04,NaN,NaN,146.68,111.24,111.47,NaN,NaN,97.87,NaN,NaN,NaN,NaN
4,13/07/19,NaN,NaN,157.55,163.85,NaN,144.87,NaN,111.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [225]:
for col in train.columns:
    train.loc[ (train[col] == 'N.S.') | (train[col] == 'N. S.') |(train[col] == 'N.S') | 
              (train[col] == 'NS') | (train[col] == 'No Sale') | 
              (train[col] == 'No sale') | (train[col] == 'N.S.\xa0') |
              (train[col] == 'NS85.07'), col] = 0

for col in test.columns:
    test.loc[test[col] == 'NS'] = 0

In [226]:
train['WeekEnding_Date'] = pd.to_datetime(train['WeekEnding_Date'])
test['WeekEnding_Date'] = pd.to_datetime(test['WeekEnding_Date'])

In [227]:
train['Day'] = train['WeekEnding_Date'].dt.day
train['Month'] = train['WeekEnding_Date'].dt.month
train['Year'] = train['WeekEnding_Date'].dt.year

test['Day'] = test['WeekEnding_Date'].dt.day
test['Month'] = test['WeekEnding_Date'].dt.month
test['Year'] = test['WeekEnding_Date'].dt.year

In [228]:
train.drop('WeekEnding_Date', axis = 1, inplace = True)
test.drop('WeekEnding_Date', axis = 1, inplace = True)

In [229]:
imputer = KNNImputer(n_neighbors=3)
imputer.fit(train)

KNNImputer(n_neighbors=3)

In [230]:
filled_test = imputer.fit_transform(test)

In [231]:
filled_test

array([[1.79026667e+02, 1.65870000e+02, 1.54080000e+02, 1.60820000e+02,
        1.34063333e+02, 1.44726667e+02, 1.16450000e+02, 1.12820000e+02,
        9.18200000e+01, 8.55600000e+01, 1.02300000e+02, 1.04323333e+02,
        8.30700000e+01, 6.53400000e+01, 0.00000000e+00, 1.50000000e+01,
        6.00000000e+00, 2.01900000e+03],
       [1.71350000e+02, 1.73120000e+02, 1.53933333e+02, 1.53010000e+02,
        1.34186667e+02, 1.47310000e+02, 1.10886667e+02, 1.15370000e+02,
        9.06800000e+01, 7.63400000e+01, 9.79866667e+01, 1.06490000e+02,
        8.13400000e+01, 7.59500000e+01, 0.00000000e+00, 2.20000000e+01,
        6.00000000e+00, 2.01900000e+03],
       [1.69663333e+02, 1.65300000e+02, 1.56140000e+02, 1.52653333e+02,
        1.32793333e+02, 1.50390000e+02, 1.10886667e+02, 1.23020000e+02,
        8.34033333e+01, 8.70733333e+01, 9.68400000e+01, 1.07350000e+02,
        6.97400000e+01, 8.69166667e+01, 0.00000000e+00, 2.90000000e+01,
        6.00000000e+00, 2.01900000e+03],
       [1.729

In [232]:
# train.fillna(-99, inplace = True)
# test.fillna(-99, inplace = True)

In [233]:
for col in train.columns:
    train[col] = pd.to_numeric(train[col])
    
# for col in test.columns:
#     test[col] = pd.to_numeric(test[col])

In [234]:
# cols = [col for col in train.columns if col not in [target, 'Day', 'Month', 'Year']]
# month = train['Month'].unique()

# for col, mon in zip(cols, month):
#     train.loc[train[col].isna() ,train['Month'] == mon, col] = train.groupby(mon)[col].mean()

In [235]:
# temp = train.groupby('Month')['Kolkata_Average_Price'].mean().to_dict()
# temp_2 = train.groupby('Year')['Kolkata_Average_Price'].mean().to_dict()

# train_dict= train.groupby(['Month','Year'])['Kolkata_Average_Price'].mean()

In [236]:
# cols = [col for col in train.columns if col not in [target, 'Day', 'Month', 'Year']]

# def mon_yr(x, y):
#     a = (temp[x]+temp[y])/2
#     return a

# for col in cols:
#     temp = train.groupby('Month')[col].mean().to_dict()
#     temp_2 = train.groupby('Year')[col].mean().to_dict()
#     test.loc[test[col].isna(),col] = test[test[col].isna()].apply(lambda x: (temp[x['Month']]+temp_2[x['Year']])/2, axis = 1)
# #     test.loc[test[col].isna(),col] = test[test[col].isna()]['Month'].apply(lambda x: temp[x])

In [237]:
train.columns

Index(['Kolkata_Average_Price', 'Kolkata_Ref_Price', 'Bangalore_Average_Price',
       'Bangalore_Ref_Price', 'Cochin_Average_Price', 'Cochin_Ref_Price',
       'Darjeeling_Average_Price', 'Darjeeling_Ref_Price',
       'Ernakulam_Average_Price', 'Ernakulam_Ref_Price',
       'Siliguri_Average_Price', 'Siliguri_Ref_Price',
       'Guwahati_Average_Price', 'Guwahati_Ref_Price', 'Average', 'Day',
       'Month', 'Year'],
      dtype='object')

In [238]:
train.isna().sum()

Kolkata_Average_Price       0
Kolkata_Ref_Price           2
Bangalore_Average_Price     0
Bangalore_Ref_Price         2
Cochin_Average_Price        0
Cochin_Ref_Price            3
Darjeeling_Average_Price    0
Darjeeling_Ref_Price        2
Ernakulam_Average_Price     0
Ernakulam_Ref_Price         1
Siliguri_Average_Price      1
Siliguri_Ref_Price          1
Guwahati_Average_Price      0
Guwahati_Ref_Price          0
Average                     0
Day                         0
Month                       0
Year                        0
dtype: int64

In [239]:
train.fillna(0, inplace = True)

In [240]:
train.isna().sum()

Kolkata_Average_Price       0
Kolkata_Ref_Price           0
Bangalore_Average_Price     0
Bangalore_Ref_Price         0
Cochin_Average_Price        0
Cochin_Ref_Price            0
Darjeeling_Average_Price    0
Darjeeling_Ref_Price        0
Ernakulam_Average_Price     0
Ernakulam_Ref_Price         0
Siliguri_Average_Price      0
Siliguri_Ref_Price          0
Guwahati_Average_Price      0
Guwahati_Ref_Price          0
Average                     0
Day                         0
Month                       0
Year                        0
dtype: int64

In [241]:
target = 'Average'

X = train.drop(target, axis = 1)
y = train[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [242]:
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_preds = lr.predict(X_test)
print(f'Error is: {np.sqrt(mean_squared_error(lr_preds, y_test))}')

# 5.149439377357231

Error is: 5.149439377357231


In [243]:
dt = DecisionTreeRegressor()
dt.fit(X_train, y_train)
dt_preds = dt.predict(X_test)
print(f'Error is: {np.sqrt(mean_squared_error(dt_preds, y_test))}')

# 3.5072325111476546

Error is: 3.2978699706547454


In [244]:
xgb = XGBRegressor()
xgb.fit(X_train, y_train)
xgb_preds = xgb.predict(X_test)
print(f'Error is: {np.sqrt(mean_squared_error(xgb_preds, y_test))}')
# 2.9496266229610777

Error is: 2.9496266229610777


In [245]:
lgb = LGBMRegressor()
lgb.fit(X_train, y_train)
lgb_preds = lgb.predict(X_test)
print(f'Error is: ' + str(np.sqrt(mean_squared_error(lgb_preds, y_test))))

# 2.554611595843692

Error is: 2.554611595843692


In [246]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
rf_preds = rf.predict(X_test)
print(f'Error is: ' + str(np.sqrt(mean_squared_error(rf_preds, y_test))))

# 2.8408485878527583

Error is: 2.789005378197836


In [247]:
cat = CatBoostRegressor()
cat.fit(X_train, y_train)
cat_preds = cat.predict(X_test)
print(f'Error is: ' + str(np.sqrt(mean_squared_error(cat_preds, y_test))))

# 2.063852222980388

Learning rate set to 0.033655
0:	learn: 13.4678153	total: 5.64ms	remaining: 5.63s
1:	learn: 13.1417935	total: 8.3ms	remaining: 4.14s
2:	learn: 12.8174925	total: 11.5ms	remaining: 3.83s
3:	learn: 12.5071142	total: 14ms	remaining: 3.49s
4:	learn: 12.2058760	total: 16.1ms	remaining: 3.2s
5:	learn: 11.9403150	total: 18.1ms	remaining: 3s
6:	learn: 11.6372244	total: 20.4ms	remaining: 2.9s
7:	learn: 11.3514234	total: 24.1ms	remaining: 2.99s
8:	learn: 11.0790003	total: 26.2ms	remaining: 2.89s
9:	learn: 10.8152588	total: 28.2ms	remaining: 2.79s
10:	learn: 10.5512969	total: 30.2ms	remaining: 2.72s
11:	learn: 10.2981363	total: 32.2ms	remaining: 2.65s
12:	learn: 10.0680017	total: 34.3ms	remaining: 2.6s
13:	learn: 9.8450758	total: 37.5ms	remaining: 2.64s
14:	learn: 9.6191041	total: 39.6ms	remaining: 2.6s
15:	learn: 9.4053050	total: 41.7ms	remaining: 2.56s
16:	learn: 9.1942994	total: 43.8ms	remaining: 2.53s
17:	learn: 8.9981736	total: 45.9ms	remaining: 2.5s
18:	learn: 8.7944417	total: 48.1ms	remaini

229:	learn: 1.0396068	total: 522ms	remaining: 1.75s
230:	learn: 1.0356994	total: 524ms	remaining: 1.75s
231:	learn: 1.0314912	total: 528ms	remaining: 1.75s
232:	learn: 1.0265460	total: 531ms	remaining: 1.75s
233:	learn: 1.0232239	total: 535ms	remaining: 1.75s
234:	learn: 1.0186787	total: 538ms	remaining: 1.75s
235:	learn: 1.0139471	total: 540ms	remaining: 1.75s
236:	learn: 1.0102696	total: 542ms	remaining: 1.75s
237:	learn: 1.0064868	total: 544ms	remaining: 1.74s
238:	learn: 1.0028521	total: 547ms	remaining: 1.74s
239:	learn: 0.9981552	total: 549ms	remaining: 1.74s
240:	learn: 0.9942920	total: 551ms	remaining: 1.74s
241:	learn: 0.9885580	total: 553ms	remaining: 1.73s
242:	learn: 0.9850839	total: 556ms	remaining: 1.73s
243:	learn: 0.9814129	total: 560ms	remaining: 1.73s
244:	learn: 0.9770019	total: 562ms	remaining: 1.73s
245:	learn: 0.9735755	total: 564ms	remaining: 1.73s
246:	learn: 0.9691768	total: 567ms	remaining: 1.73s
247:	learn: 0.9653909	total: 569ms	remaining: 1.73s
248:	learn: 

389:	learn: 0.5874639	total: 870ms	remaining: 1.36s
390:	learn: 0.5860218	total: 874ms	remaining: 1.36s
391:	learn: 0.5841465	total: 877ms	remaining: 1.36s
392:	learn: 0.5825580	total: 879ms	remaining: 1.36s
393:	learn: 0.5818330	total: 882ms	remaining: 1.36s
394:	learn: 0.5806466	total: 885ms	remaining: 1.35s
395:	learn: 0.5786281	total: 887ms	remaining: 1.35s
396:	learn: 0.5761458	total: 890ms	remaining: 1.35s
397:	learn: 0.5741574	total: 893ms	remaining: 1.35s
398:	learn: 0.5720893	total: 896ms	remaining: 1.35s
399:	learn: 0.5709130	total: 899ms	remaining: 1.35s
400:	learn: 0.5697490	total: 903ms	remaining: 1.35s
401:	learn: 0.5677725	total: 905ms	remaining: 1.35s
402:	learn: 0.5655554	total: 908ms	remaining: 1.34s
403:	learn: 0.5637377	total: 910ms	remaining: 1.34s
404:	learn: 0.5622931	total: 912ms	remaining: 1.34s
405:	learn: 0.5597086	total: 914ms	remaining: 1.34s
406:	learn: 0.5577355	total: 916ms	remaining: 1.33s
407:	learn: 0.5556282	total: 918ms	remaining: 1.33s
408:	learn: 

548:	learn: 0.3720576	total: 1.22s	remaining: 1s
549:	learn: 0.3709658	total: 1.22s	remaining: 1s
550:	learn: 0.3700193	total: 1.23s	remaining: 999ms
551:	learn: 0.3692429	total: 1.23s	remaining: 997ms
552:	learn: 0.3684848	total: 1.23s	remaining: 995ms
553:	learn: 0.3678319	total: 1.23s	remaining: 993ms
554:	learn: 0.3670121	total: 1.24s	remaining: 990ms
555:	learn: 0.3664697	total: 1.24s	remaining: 988ms
556:	learn: 0.3658116	total: 1.24s	remaining: 986ms
557:	learn: 0.3647361	total: 1.24s	remaining: 983ms
558:	learn: 0.3638555	total: 1.24s	remaining: 981ms
559:	learn: 0.3633350	total: 1.25s	remaining: 979ms
560:	learn: 0.3627302	total: 1.25s	remaining: 977ms
561:	learn: 0.3616892	total: 1.25s	remaining: 976ms
562:	learn: 0.3605028	total: 1.25s	remaining: 974ms
563:	learn: 0.3594868	total: 1.26s	remaining: 972ms
564:	learn: 0.3586139	total: 1.26s	remaining: 969ms
565:	learn: 0.3578195	total: 1.26s	remaining: 967ms
566:	learn: 0.3570810	total: 1.26s	remaining: 964ms
567:	learn: 0.3565

710:	learn: 0.2602335	total: 1.57s	remaining: 639ms
711:	learn: 0.2596432	total: 1.57s	remaining: 637ms
712:	learn: 0.2594874	total: 1.58s	remaining: 635ms
713:	learn: 0.2585867	total: 1.58s	remaining: 633ms
714:	learn: 0.2580543	total: 1.58s	remaining: 630ms
715:	learn: 0.2574845	total: 1.58s	remaining: 628ms
716:	learn: 0.2570985	total: 1.59s	remaining: 626ms
717:	learn: 0.2564639	total: 1.59s	remaining: 624ms
718:	learn: 0.2557313	total: 1.59s	remaining: 622ms
719:	learn: 0.2554819	total: 1.59s	remaining: 620ms
720:	learn: 0.2546635	total: 1.6s	remaining: 618ms
721:	learn: 0.2542216	total: 1.6s	remaining: 616ms
722:	learn: 0.2541447	total: 1.6s	remaining: 613ms
723:	learn: 0.2535475	total: 1.6s	remaining: 612ms
724:	learn: 0.2531000	total: 1.61s	remaining: 610ms
725:	learn: 0.2525924	total: 1.61s	remaining: 608ms
726:	learn: 0.2521298	total: 1.61s	remaining: 606ms
727:	learn: 0.2515226	total: 1.61s	remaining: 603ms
728:	learn: 0.2509511	total: 1.62s	remaining: 601ms
729:	learn: 0.25

868:	learn: 0.1913172	total: 1.92s	remaining: 289ms
869:	learn: 0.1906797	total: 1.92s	remaining: 287ms
870:	learn: 0.1901788	total: 1.92s	remaining: 285ms
871:	learn: 0.1895208	total: 1.93s	remaining: 283ms
872:	learn: 0.1891096	total: 1.93s	remaining: 281ms
873:	learn: 0.1886624	total: 1.93s	remaining: 278ms
874:	learn: 0.1882670	total: 1.93s	remaining: 276ms
875:	learn: 0.1881934	total: 1.94s	remaining: 274ms
876:	learn: 0.1876713	total: 1.94s	remaining: 272ms
877:	learn: 0.1873516	total: 1.94s	remaining: 270ms
878:	learn: 0.1868391	total: 1.94s	remaining: 267ms
879:	learn: 0.1865817	total: 1.94s	remaining: 265ms
880:	learn: 0.1861271	total: 1.95s	remaining: 263ms
881:	learn: 0.1859896	total: 1.95s	remaining: 261ms
882:	learn: 0.1856619	total: 1.95s	remaining: 259ms
883:	learn: 0.1853325	total: 1.96s	remaining: 257ms
884:	learn: 0.1850049	total: 1.96s	remaining: 254ms
885:	learn: 0.1846284	total: 1.96s	remaining: 252ms
886:	learn: 0.1841810	total: 1.96s	remaining: 250ms
887:	learn: 

In [84]:
ft = cat.predict(filled_test)

In [248]:
test_1 = pd.DataFrame(filled_test, columns = train.columns)

In [249]:
test_1.head()

,Kolkata_Average_Price,Kolkata_Ref_Price,Bangalore_Average_Price,Bangalore_Ref_Price,Cochin_Average_Price,Cochin_Ref_Price,Darjeeling_Average_Price,Darjeeling_Ref_Price,Ernakulam_Average_Price,Ernakulam_Ref_Price,Siliguri_Average_Price,Siliguri_Ref_Price,Guwahati_Average_Price,Guwahati_Ref_Price,Average,Day,Month,Year
0,179.026667,165.870000,154.080000,160.820000,134.063333,144.726667,116.450000,112.82,91.820000,85.560000,102.300000,104.323333,83.070000,65.340000,0.0,15.0,6.0,2019.0
1,171.350000,173.120000,153.933333,153.010000,134.186667,147.310000,110.886667,115.37,90.680000,76.340000,97.986667,106.490000,81.340000,75.950000,0.0,22.0,6.0,2019.0
2,169.663333,165.300000,156.140000,152.653333,132.793333,150.390000,110.886667,123.02,83.403333,87.073333,96.840000,107.350000,69.740000,86.916667,0.0,29.0,6.0,2019.0
3,172.973333,161.926667,158.040000,161.760000,133.913333,146.680000,111.240000,111.47,83.193333,91.413333,97.870000,106.353333,76.583333,80.293333,0.0,7.0,6.0,2019.0
4,181.733333,161.926667,157.550000,163.850000,134.063333,144.870000,113.573333,111.44,82.570000,91.413333,97.596667,105.710000,75.976667,80.823333,0.0,13.0,7.0,2019.0


## Cross Validation

In [250]:
def cross_val(train, test, features, clf):
    preds_df = pd.DataFrame()
    oofs = np.zeros(len(train))
    preds = np.zeros(len(test))
    
    folds = KFold(n_splits = 10, shuffle = True, random_state = 0)
    
    for index, (trn_idx, val_idx) in enumerate(folds.split(train)):
        print()
        print(f'*******************Fold {index+1}*******************')
        print()
        X_trn, y_trn = train[features].iloc[trn_idx], train[target].iloc[trn_idx]
        X_val, y_val = train[features].iloc[val_idx], train[target].iloc[val_idx]
    
        X_test = test[features]
        
        _ = clf.fit(X_trn, y_trn)
        
        val_preds = clf.predict(X_val)
        test_preds = clf.predict(X_test)
        
        oofs[val_idx] = val_preds
        preds_df[str(index+1)] = test_preds
        
        rmse = np.sqrt(mean_squared_error(y_val, val_preds))
        print('Mean Squared Error is : ',rmse)
        
    total_score = np.sqrt(mean_squared_error(train[target], oofs))
    print('Total score :',total_score)
    preds_df['mean'] = preds_df.apply(lambda x:x.mean(), axis = 1)
    preds = preds_df['mean']
    return oofs, preds

In [295]:
features = [col for col in train.columns if col not in [target]]

cat_oofs, cat_preds = cross_val(train, test_1, features, CatBoostRegressor(random_state = 0))
 
# 1.8620398667629492


*******************Fold 1*******************

Learning rate set to 0.034364
0:	learn: 13.4075625	total: 4.79ms	remaining: 4.78s
1:	learn: 13.0785571	total: 7.17ms	remaining: 3.58s
2:	learn: 12.7333336	total: 9.38ms	remaining: 3.12s
3:	learn: 12.4164089	total: 11.8ms	remaining: 2.94s
4:	learn: 12.1047783	total: 14ms	remaining: 2.79s
5:	learn: 11.8272856	total: 16.7ms	remaining: 2.76s
6:	learn: 11.5318389	total: 19ms	remaining: 2.7s
7:	learn: 11.2440805	total: 22.3ms	remaining: 2.77s
8:	learn: 10.9838739	total: 24.6ms	remaining: 2.71s
9:	learn: 10.7296105	total: 27ms	remaining: 2.67s
10:	learn: 10.4639604	total: 29.3ms	remaining: 2.63s
11:	learn: 10.2253631	total: 31.6ms	remaining: 2.6s
12:	learn: 9.9791877	total: 34.5ms	remaining: 2.62s
13:	learn: 9.7473037	total: 38.5ms	remaining: 2.71s
14:	learn: 9.5263418	total: 42.8ms	remaining: 2.81s
15:	learn: 9.3121259	total: 46.4ms	remaining: 2.85s
16:	learn: 9.0963981	total: 49.6ms	remaining: 2.87s
17:	learn: 8.8935717	total: 52.6ms	remaining:

199:	learn: 1.1980413	total: 462ms	remaining: 1.85s
200:	learn: 1.1938723	total: 465ms	remaining: 1.85s
201:	learn: 1.1904176	total: 467ms	remaining: 1.84s
202:	learn: 1.1867556	total: 469ms	remaining: 1.84s
203:	learn: 1.1808961	total: 471ms	remaining: 1.84s
204:	learn: 1.1762848	total: 474ms	remaining: 1.84s
205:	learn: 1.1721253	total: 476ms	remaining: 1.83s
206:	learn: 1.1674628	total: 478ms	remaining: 1.83s
207:	learn: 1.1613359	total: 480ms	remaining: 1.83s
208:	learn: 1.1557110	total: 483ms	remaining: 1.83s
209:	learn: 1.1500662	total: 485ms	remaining: 1.82s
210:	learn: 1.1455930	total: 487ms	remaining: 1.82s
211:	learn: 1.1403475	total: 491ms	remaining: 1.82s
212:	learn: 1.1347998	total: 494ms	remaining: 1.82s
213:	learn: 1.1302108	total: 496ms	remaining: 1.82s
214:	learn: 1.1238906	total: 499ms	remaining: 1.82s
215:	learn: 1.1190333	total: 501ms	remaining: 1.82s
216:	learn: 1.1148472	total: 503ms	remaining: 1.81s
217:	learn: 1.1117750	total: 505ms	remaining: 1.81s
218:	learn: 

362:	learn: 0.6692618	total: 813ms	remaining: 1.43s
363:	learn: 0.6683545	total: 815ms	remaining: 1.42s
364:	learn: 0.6657832	total: 817ms	remaining: 1.42s
365:	learn: 0.6638434	total: 820ms	remaining: 1.42s
366:	learn: 0.6615830	total: 822ms	remaining: 1.42s
367:	learn: 0.6600296	total: 824ms	remaining: 1.41s
368:	learn: 0.6581591	total: 826ms	remaining: 1.41s
369:	learn: 0.6568998	total: 828ms	remaining: 1.41s
370:	learn: 0.6549601	total: 831ms	remaining: 1.41s
371:	learn: 0.6531588	total: 833ms	remaining: 1.41s
372:	learn: 0.6514284	total: 835ms	remaining: 1.4s
373:	learn: 0.6484106	total: 837ms	remaining: 1.4s
374:	learn: 0.6476493	total: 840ms	remaining: 1.4s
375:	learn: 0.6451484	total: 842ms	remaining: 1.4s
376:	learn: 0.6430167	total: 845ms	remaining: 1.4s
377:	learn: 0.6405907	total: 849ms	remaining: 1.4s
378:	learn: 0.6387079	total: 852ms	remaining: 1.4s
379:	learn: 0.6373996	total: 854ms	remaining: 1.39s
380:	learn: 0.6340563	total: 858ms	remaining: 1.39s
381:	learn: 0.63331

599:	learn: 0.3607902	total: 1.34s	remaining: 892ms
600:	learn: 0.3596977	total: 1.34s	remaining: 890ms
601:	learn: 0.3587634	total: 1.34s	remaining: 888ms
602:	learn: 0.3582115	total: 1.34s	remaining: 885ms
603:	learn: 0.3570511	total: 1.35s	remaining: 883ms
604:	learn: 0.3564597	total: 1.35s	remaining: 881ms
605:	learn: 0.3556589	total: 1.35s	remaining: 878ms
606:	learn: 0.3547039	total: 1.35s	remaining: 876ms
607:	learn: 0.3543313	total: 1.35s	remaining: 874ms
608:	learn: 0.3533926	total: 1.36s	remaining: 872ms
609:	learn: 0.3524154	total: 1.36s	remaining: 869ms
610:	learn: 0.3513876	total: 1.36s	remaining: 867ms
611:	learn: 0.3506129	total: 1.36s	remaining: 865ms
612:	learn: 0.3499216	total: 1.37s	remaining: 863ms
613:	learn: 0.3490544	total: 1.37s	remaining: 861ms
614:	learn: 0.3482172	total: 1.37s	remaining: 859ms
615:	learn: 0.3471019	total: 1.37s	remaining: 857ms
616:	learn: 0.3462289	total: 1.38s	remaining: 854ms
617:	learn: 0.3454309	total: 1.38s	remaining: 852ms
618:	learn: 

765:	learn: 0.2480220	total: 1.69s	remaining: 516ms
766:	learn: 0.2476436	total: 1.69s	remaining: 514ms
767:	learn: 0.2468645	total: 1.69s	remaining: 512ms
768:	learn: 0.2462759	total: 1.7s	remaining: 510ms
769:	learn: 0.2456590	total: 1.7s	remaining: 507ms
770:	learn: 0.2451149	total: 1.7s	remaining: 505ms
771:	learn: 0.2442192	total: 1.7s	remaining: 503ms
772:	learn: 0.2438064	total: 1.71s	remaining: 501ms
773:	learn: 0.2429862	total: 1.71s	remaining: 499ms
774:	learn: 0.2424497	total: 1.71s	remaining: 496ms
775:	learn: 0.2419622	total: 1.71s	remaining: 494ms
776:	learn: 0.2415844	total: 1.72s	remaining: 492ms
777:	learn: 0.2407221	total: 1.72s	remaining: 490ms
778:	learn: 0.2403089	total: 1.72s	remaining: 488ms
779:	learn: 0.2399698	total: 1.72s	remaining: 486ms
780:	learn: 0.2396312	total: 1.72s	remaining: 483ms
781:	learn: 0.2390292	total: 1.73s	remaining: 481ms
782:	learn: 0.2386009	total: 1.73s	remaining: 479ms
783:	learn: 0.2381831	total: 1.73s	remaining: 477ms
784:	learn: 0.23

...

160:	learn: 1.4241826	total: 469ms	remaining: 2.44s
161:	learn: 1.4167388	total: 472ms	remaining: 2.44s
162:	learn: 1.4078118	total: 475ms	remaining: 2.44s
163:	learn: 1.4014917	total: 478ms	remaining: 2.44s
164:	learn: 1.3952136	total: 481ms	remaining: 2.44s
165:	learn: 1.3884564	total: 484ms	remaining: 2.43s
166:	learn: 1.3819035	total: 487ms	remaining: 2.43s
167:	learn: 1.3756702	total: 490ms	remaining: 2.43s
168:	learn: 1.3708238	total: 493ms	remaining: 2.42s
169:	learn: 1.3644079	total: 496ms	remaining: 2.42s
170:	learn: 1.3579935	total: 499ms	remaining: 2.42s
171:	learn: 1.3524930	total: 503ms	remaining: 2.42s
172:	learn: 1.3478228	total: 506ms	remaining: 2.42s
173:	learn: 1.3410577	total: 510ms	remaining: 2.42s
174:	learn: 1.3359584	total: 513ms	remaining: 2.42s
175:	learn: 1.3300794	total: 516ms	remaining: 2.41s
176:	learn: 1.3236967	total: 518ms	remaining: 2.41s
177:	learn: 1.3192747	total: 521ms	remaining: 2.4s
178:	learn: 1.3128549	total: 523ms	remaining: 2.4s
179:	learn: 1.

344:	learn: 0.7194568	total: 1s	remaining: 1.9s
345:	learn: 0.7163951	total: 1s	remaining: 1.9s
346:	learn: 0.7143211	total: 1.01s	remaining: 1.89s
347:	learn: 0.7130107	total: 1.01s	remaining: 1.89s
348:	learn: 0.7103665	total: 1.01s	remaining: 1.89s
349:	learn: 0.7074682	total: 1.01s	remaining: 1.88s
350:	learn: 0.7044566	total: 1.02s	remaining: 1.88s
351:	learn: 0.7020953	total: 1.02s	remaining: 1.88s
352:	learn: 0.6997756	total: 1.02s	remaining: 1.88s
353:	learn: 0.6970544	total: 1.03s	remaining: 1.87s
354:	learn: 0.6936401	total: 1.03s	remaining: 1.87s
355:	learn: 0.6926118	total: 1.03s	remaining: 1.87s
356:	learn: 0.6902862	total: 1.04s	remaining: 1.87s
357:	learn: 0.6897052	total: 1.04s	remaining: 1.86s
358:	learn: 0.6879519	total: 1.04s	remaining: 1.86s
359:	learn: 0.6856089	total: 1.05s	remaining: 1.86s
360:	learn: 0.6831082	total: 1.05s	remaining: 1.86s
361:	learn: 0.6805613	total: 1.05s	remaining: 1.86s
362:	learn: 0.6791082	total: 1.06s	remaining: 1.85s
363:	learn: 0.677030

530:	learn: 0.4195925	total: 1.52s	remaining: 1.34s
531:	learn: 0.4189770	total: 1.53s	remaining: 1.34s
532:	learn: 0.4182583	total: 1.53s	remaining: 1.34s
533:	learn: 0.4175772	total: 1.53s	remaining: 1.34s
534:	learn: 0.4166045	total: 1.53s	remaining: 1.33s
535:	learn: 0.4150709	total: 1.54s	remaining: 1.33s
536:	learn: 0.4140595	total: 1.54s	remaining: 1.33s
537:	learn: 0.4128484	total: 1.54s	remaining: 1.33s
538:	learn: 0.4116508	total: 1.55s	remaining: 1.32s
539:	learn: 0.4108244	total: 1.55s	remaining: 1.32s
540:	learn: 0.4098786	total: 1.55s	remaining: 1.32s
541:	learn: 0.4087063	total: 1.56s	remaining: 1.31s
542:	learn: 0.4075345	total: 1.56s	remaining: 1.31s
543:	learn: 0.4066438	total: 1.56s	remaining: 1.31s
544:	learn: 0.4052115	total: 1.57s	remaining: 1.31s
545:	learn: 0.4039203	total: 1.57s	remaining: 1.3s
546:	learn: 0.4025789	total: 1.57s	remaining: 1.3s
547:	learn: 0.4014517	total: 1.57s	remaining: 1.3s
548:	learn: 0.4004953	total: 1.58s	remaining: 1.3s
549:	learn: 0.39

719:	learn: 0.2641124	total: 2.05s	remaining: 798ms
720:	learn: 0.2636175	total: 2.05s	remaining: 795ms
721:	learn: 0.2627610	total: 2.06s	remaining: 792ms
722:	learn: 0.2620323	total: 2.06s	remaining: 789ms
723:	learn: 0.2614567	total: 2.06s	remaining: 786ms
724:	learn: 0.2609385	total: 2.06s	remaining: 783ms
725:	learn: 0.2602792	total: 2.07s	remaining: 781ms
726:	learn: 0.2596874	total: 2.07s	remaining: 778ms
727:	learn: 0.2590346	total: 2.07s	remaining: 775ms
728:	learn: 0.2584641	total: 2.08s	remaining: 772ms
729:	learn: 0.2578526	total: 2.08s	remaining: 770ms
730:	learn: 0.2573427	total: 2.09s	remaining: 768ms
731:	learn: 0.2568007	total: 2.09s	remaining: 765ms
732:	learn: 0.2561281	total: 2.09s	remaining: 763ms
733:	learn: 0.2555773	total: 2.1s	remaining: 760ms
734:	learn: 0.2551637	total: 2.1s	remaining: 758ms
735:	learn: 0.2545987	total: 2.1s	remaining: 755ms
736:	learn: 0.2539704	total: 2.11s	remaining: 753ms
737:	learn: 0.2535171	total: 2.11s	remaining: 750ms
738:	learn: 0.2

894:	learn: 0.1790512	total: 2.58s	remaining: 303ms
895:	learn: 0.1786988	total: 2.58s	remaining: 300ms
896:	learn: 0.1784192	total: 2.59s	remaining: 297ms
897:	learn: 0.1780182	total: 2.59s	remaining: 294ms
898:	learn: 0.1776754	total: 2.59s	remaining: 291ms
899:	learn: 0.1774724	total: 2.6s	remaining: 288ms
900:	learn: 0.1770885	total: 2.6s	remaining: 286ms
901:	learn: 0.1768137	total: 2.6s	remaining: 283ms
902:	learn: 0.1764427	total: 2.61s	remaining: 280ms
903:	learn: 0.1760268	total: 2.61s	remaining: 277ms
904:	learn: 0.1756780	total: 2.61s	remaining: 274ms
905:	learn: 0.1752930	total: 2.62s	remaining: 271ms
906:	learn: 0.1749599	total: 2.62s	remaining: 268ms
907:	learn: 0.1747714	total: 2.62s	remaining: 266ms
908:	learn: 0.1744007	total: 2.63s	remaining: 263ms
909:	learn: 0.1740253	total: 2.63s	remaining: 260ms
910:	learn: 0.1735970	total: 2.64s	remaining: 258ms
911:	learn: 0.1733336	total: 2.64s	remaining: 255ms
912:	learn: 0.1728723	total: 2.64s	remaining: 252ms
913:	learn: 0.1

In [297]:
xgb_oofs, xgb_preds = cross_val(train, test_1, features, XGBRegressor(**xgb_params, random_state = 0))

# 2.4574167322150546
# 2.1775464374755176


*******************Fold 1*******************

Mean Squared Error is :  1.7844267318146159

*******************Fold 2*******************

Mean Squared Error is :  2.4149336398202124

*******************Fold 3*******************

Mean Squared Error is :  2.8653598283084047

*******************Fold 4*******************

Mean Squared Error is :  1.5285792671797533

*******************Fold 5*******************

Mean Squared Error is :  2.161625926697176

*******************Fold 6*******************

Mean Squared Error is :  2.952540683103814

*******************Fold 7*******************

Mean Squared Error is :  1.8816197692363312

*******************Fold 8*******************

Mean Squared Error is :  1.90478911260242

*******************Fold 9*******************

Mean Squared Error is :  1.4008960785305629

*******************Fold 10*******************

Mean Squared Error is :  2.3068970311691714
Total score : 2.1775464374755176


In [298]:
lgb_oofs, lgb_preds = cross_val(train, test_1, features, LGBMRegressor(**lgb_params, random_state = 0))

# 2.524165556394003
# 2.1883648779042204


*******************Fold 1*******************

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Mean Squared Error is :  1.9404428485120886

*******************Fold 2*******************

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Mean Squared Error is :  1.7587683841774464

*******************Fold 3*******************

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Mean Squared Error is :  2.8848084617214895

*******************Fold 4*******************

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Mean Squared Error is :  1.495209041977584

*******************Fold 5*******************

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly s

In [299]:
rf_oofs, rf_preds = cross_val(train, test_1, features, RandomForestRegressor(random_state = 0))

# 3.0191970301185536
# 3.760688920526766


*******************Fold 1*******************

Mean Squared Error is :  2.775722096129037

*******************Fold 2*******************

Mean Squared Error is :  2.9227914845611047

*******************Fold 3*******************

Mean Squared Error is :  3.9844600019638436

*******************Fold 4*******************

Mean Squared Error is :  2.1042262164466936

*******************Fold 5*******************

Mean Squared Error is :  1.7966792603697734

*******************Fold 6*******************

Mean Squared Error is :  5.099846663661177

*******************Fold 7*******************

Mean Squared Error is :  3.245397560316175

*******************Fold 8*******************

Mean Squared Error is :  2.175499937158081

*******************Fold 9*******************

Mean Squared Error is :  1.5653330917580814

*******************Fold 10*******************

Mean Squared Error is :  2.765983374514601
Total score : 3.0191970301185536


In [ ]:
# new_preds = (lgb_preds+xgb_preds+rf_preds+cat_preds)/4

---

## Stacking

In [300]:
new_train = pd.DataFrame()
new_test = pd.DataFrame()

new_train['lgb'] = lgb_oofs
new_train['xgb'] = xgb_oofs
new_train['cat'] = cat_oofs
new_train['rf'] = rf_oofs
new_train[target] = train[target]

new_test['lgb'] = lgb_preds
new_test['xgb'] = xgb_preds
new_test['cat'] = cat_preds
new_test['rf'] = rf_preds

In [301]:
new_features = [col for col in new_train if col not in [target]]

new_rf_oofs, new_rf_preds = cross_val(new_train, new_test, new_features, RandomForestRegressor(random_state = 0))
# 2.1459312421903807
# 2.0350886761106466


*******************Fold 1*******************

Mean Squared Error is :  1.915651243726618

*******************Fold 2*******************

Mean Squared Error is :  2.0543769580910864

*******************Fold 3*******************

Mean Squared Error is :  2.6427752966609632

*******************Fold 4*******************

Mean Squared Error is :  1.224390849530553

*******************Fold 5*******************

Mean Squared Error is :  1.5166970735490721

*******************Fold 6*******************

Mean Squared Error is :  2.909328104583904

*******************Fold 7*******************

Mean Squared Error is :  1.2024927743834013

*******************Fold 8*******************

Mean Squared Error is :  1.4683075584830276

*******************Fold 9*******************

Mean Squared Error is :  1.2084736152560993

*******************Fold 10*******************

Mean Squared Error is :  2.060957070302354
Total score : 1.9099997895054706


In [302]:
new_lgb_oofs, new_lgb_preds = cross_val(new_train, new_test, new_features, LGBMRegressor(random_state = 0))

# 2.306473742653118
# 2.215145596842139


*******************Fold 1*******************

Mean Squared Error is :  2.075044608008886

*******************Fold 2*******************

Mean Squared Error is :  2.249716639862059

*******************Fold 3*******************

Mean Squared Error is :  3.125516904417568

*******************Fold 4*******************

Mean Squared Error is :  1.6289265300345446

*******************Fold 5*******************

Mean Squared Error is :  1.6050727394976039

*******************Fold 6*******************

Mean Squared Error is :  3.1862538149646946

*******************Fold 7*******************

Mean Squared Error is :  1.734433862008127

*******************Fold 8*******************

Mean Squared Error is :  1.552345211854061

*******************Fold 9*******************

Mean Squared Error is :  1.308341067341952

*******************Fold 10*******************

Mean Squared Error is :  2.1913850073388583
Total score : 2.1565880914655717


In [303]:
new_xgb_oofs, new_xgb_preds = cross_val(new_train, new_test, new_features, XGBRegressor(random_state = 0))

# 2.594439419055631
# 2.2621072751989777


*******************Fold 1*******************

Mean Squared Error is :  2.130089218914072

*******************Fold 2*******************

Mean Squared Error is :  2.433577841746409

*******************Fold 3*******************

Mean Squared Error is :  3.0273567908593706

*******************Fold 4*******************

Mean Squared Error is :  1.2067904933587408

*******************Fold 5*******************

Mean Squared Error is :  1.5509241111038101

*******************Fold 6*******************

Mean Squared Error is :  2.7343035383941543

*******************Fold 7*******************

Mean Squared Error is :  1.5196501299104102

*******************Fold 8*******************

Mean Squared Error is :  1.552073018509728

*******************Fold 9*******************

Mean Squared Error is :  1.3518607658080524

*******************Fold 10*******************

Mean Squared Error is :  2.2282825537595223
Total score : 2.0627370588267464


In [304]:
new_cat_oofs, new_cat_preds = cross_val(new_train, new_test, new_features, CatBoostRegressor(random_state = 0))

# 2.2009074543956393
# 2.0913849881824556


*******************Fold 1*******************

Learning rate set to 0.034364
0:	learn: 13.3170665	total: 1.58ms	remaining: 1.58s
1:	learn: 12.9140057	total: 2.96ms	remaining: 1.48s
2:	learn: 12.5284564	total: 4.5ms	remaining: 1.5s
3:	learn: 12.1607000	total: 5.97ms	remaining: 1.49s
4:	learn: 11.7903100	total: 7.38ms	remaining: 1.47s
5:	learn: 11.4379911	total: 8.94ms	remaining: 1.48s
6:	learn: 11.0996388	total: 10.4ms	remaining: 1.48s
7:	learn: 10.7743222	total: 11.8ms	remaining: 1.46s
8:	learn: 10.4506252	total: 13.2ms	remaining: 1.46s
9:	learn: 10.1455644	total: 15.5ms	remaining: 1.53s
10:	learn: 9.8546126	total: 17.2ms	remaining: 1.55s
11:	learn: 9.5704258	total: 18.5ms	remaining: 1.53s
12:	learn: 9.3021197	total: 19.9ms	remaining: 1.51s
13:	learn: 9.0290140	total: 21.2ms	remaining: 1.49s
14:	learn: 8.7754909	total: 22.5ms	remaining: 1.48s
15:	learn: 8.5328248	total: 23.8ms	remaining: 1.46s
16:	learn: 8.2864653	total: 25.1ms	remaining: 1.45s
17:	learn: 8.0572856	total: 26.4ms	remain

206:	learn: 1.5053283	total: 278ms	remaining: 1.06s
207:	learn: 1.5039143	total: 280ms	remaining: 1.07s
208:	learn: 1.5018913	total: 281ms	remaining: 1.06s
209:	learn: 1.5001819	total: 283ms	remaining: 1.06s
210:	learn: 1.4991985	total: 284ms	remaining: 1.06s
211:	learn: 1.4979846	total: 285ms	remaining: 1.06s
212:	learn: 1.4960158	total: 286ms	remaining: 1.06s
213:	learn: 1.4937808	total: 288ms	remaining: 1.06s
214:	learn: 1.4916816	total: 289ms	remaining: 1.05s
215:	learn: 1.4890390	total: 290ms	remaining: 1.05s
216:	learn: 1.4877760	total: 291ms	remaining: 1.05s
217:	learn: 1.4861153	total: 293ms	remaining: 1.05s
218:	learn: 1.4844154	total: 294ms	remaining: 1.05s
219:	learn: 1.4836156	total: 295ms	remaining: 1.04s
220:	learn: 1.4830500	total: 296ms	remaining: 1.04s
221:	learn: 1.4822289	total: 297ms	remaining: 1.04s
222:	learn: 1.4802698	total: 299ms	remaining: 1.04s
223:	learn: 1.4789143	total: 300ms	remaining: 1.04s
224:	learn: 1.4785162	total: 301ms	remaining: 1.04s
225:	learn: 

502:	learn: 1.1870673	total: 624ms	remaining: 617ms
503:	learn: 1.1867973	total: 626ms	remaining: 616ms
504:	learn: 1.1850926	total: 627ms	remaining: 615ms
505:	learn: 1.1847115	total: 628ms	remaining: 613ms
506:	learn: 1.1836387	total: 630ms	remaining: 612ms
507:	learn: 1.1825667	total: 631ms	remaining: 611ms
508:	learn: 1.1825155	total: 632ms	remaining: 609ms
509:	learn: 1.1824282	total: 633ms	remaining: 608ms
510:	learn: 1.1804518	total: 634ms	remaining: 607ms
511:	learn: 1.1784849	total: 635ms	remaining: 606ms
512:	learn: 1.1774467	total: 637ms	remaining: 604ms
513:	learn: 1.1773985	total: 638ms	remaining: 603ms
514:	learn: 1.1768316	total: 639ms	remaining: 602ms
515:	learn: 1.1748043	total: 641ms	remaining: 601ms
516:	learn: 1.1745729	total: 642ms	remaining: 600ms
517:	learn: 1.1734992	total: 643ms	remaining: 598ms
518:	learn: 1.1724167	total: 644ms	remaining: 597ms
519:	learn: 1.1710069	total: 646ms	remaining: 596ms
520:	learn: 1.1697403	total: 647ms	remaining: 595ms
521:	learn: 

798:	learn: 0.9226750	total: 972ms	remaining: 245ms
799:	learn: 0.9217428	total: 974ms	remaining: 244ms
800:	learn: 0.9208291	total: 976ms	remaining: 242ms
801:	learn: 0.9194203	total: 977ms	remaining: 241ms
802:	learn: 0.9186135	total: 978ms	remaining: 240ms
803:	learn: 0.9185486	total: 979ms	remaining: 239ms
804:	learn: 0.9168572	total: 980ms	remaining: 237ms
805:	learn: 0.9157076	total: 982ms	remaining: 236ms
806:	learn: 0.9148967	total: 983ms	remaining: 235ms
807:	learn: 0.9130482	total: 984ms	remaining: 234ms
808:	learn: 0.9130010	total: 985ms	remaining: 233ms
809:	learn: 0.9129348	total: 986ms	remaining: 231ms
810:	learn: 0.9122213	total: 988ms	remaining: 230ms
811:	learn: 0.9119730	total: 989ms	remaining: 229ms
812:	learn: 0.9103119	total: 990ms	remaining: 228ms
813:	learn: 0.9097001	total: 991ms	remaining: 226ms
814:	learn: 0.9089581	total: 992ms	remaining: 225ms
815:	learn: 0.9081541	total: 994ms	remaining: 224ms
816:	learn: 0.9067348	total: 995ms	remaining: 223ms
817:	learn: 

Mean Squared Error is :  1.8665153345131782

*******************Fold 2*******************

Learning rate set to 0.034364
0:	learn: 13.3460576	total: 1.61ms	remaining: 1.61s
1:	learn: 12.9427918	total: 3.1ms	remaining: 1.54s
2:	learn: 12.5615279	total: 4.41ms	remaining: 1.46s
3:	learn: 12.1908630	total: 5.71ms	remaining: 1.42s
4:	learn: 11.8208725	total: 6.95ms	remaining: 1.38s
5:	learn: 11.4658843	total: 8.31ms	remaining: 1.38s
6:	learn: 11.1361553	total: 9.62ms	remaining: 1.36s
7:	learn: 10.8106012	total: 10.9ms	remaining: 1.35s
8:	learn: 10.4867839	total: 12.2ms	remaining: 1.34s
9:	learn: 10.1878255	total: 13.8ms	remaining: 1.37s
10:	learn: 9.8971249	total: 16ms	remaining: 1.44s
11:	learn: 9.6151669	total: 17.3ms	remaining: 1.43s
12:	learn: 9.3403531	total: 18.8ms	remaining: 1.43s
13:	learn: 9.0604266	total: 20.2ms	remaining: 1.42s
14:	learn: 8.8057260	total: 21.5ms	remaining: 1.41s
15:	learn: 8.5556229	total: 23ms	remaining: 1.41s
16:	learn: 8.3178273	total: 24.5ms	remaining: 1.42s


...

Mean Squared Error is :  1.2576625804307737

*******************Fold 10*******************

Learning rate set to 0.034376
0:	learn: 13.2748955	total: 1.97ms	remaining: 1.97s
1:	learn: 12.8728876	total: 3.35ms	remaining: 1.67s
2:	learn: 12.4854576	total: 4.7ms	remaining: 1.56s
3:	learn: 12.1233438	total: 5.91ms	remaining: 1.47s
4:	learn: 11.7549031	total: 7.22ms	remaining: 1.44s
5:	learn: 11.4033895	total: 8.55ms	remaining: 1.42s
6:	learn: 11.0582816	total: 9.9ms	remaining: 1.4s
7:	learn: 10.7363864	total: 11.2ms	remaining: 1.39s
8:	learn: 10.4132897	total: 12.5ms	remaining: 1.37s
9:	learn: 10.1163948	total: 13.7ms	remaining: 1.35s
10:	learn: 9.8274751	total: 14.9ms	remaining: 1.34s
11:	learn: 9.5447316	total: 16.4ms	remaining: 1.35s
12:	learn: 9.2778235	total: 17.7ms	remaining: 1.35s
13:	learn: 9.0003691	total: 19ms	remaining: 1.34s
14:	learn: 8.7492039	total: 20.4ms	remaining: 1.34s
15:	learn: 8.5001696	total: 21.6ms	remaining: 1.33s
16:	learn: 8.2533510	total: 23ms	remaining: 1.33s
1

207:	learn: 1.4813691	total: 287ms	remaining: 1.09s
208:	learn: 1.4801161	total: 289ms	remaining: 1.09s
209:	learn: 1.4791068	total: 290ms	remaining: 1.09s
210:	learn: 1.4776188	total: 291ms	remaining: 1.09s
211:	learn: 1.4751634	total: 293ms	remaining: 1.09s
212:	learn: 1.4746468	total: 294ms	remaining: 1.09s
213:	learn: 1.4708891	total: 296ms	remaining: 1.08s
214:	learn: 1.4687989	total: 297ms	remaining: 1.08s
215:	learn: 1.4661627	total: 298ms	remaining: 1.08s
216:	learn: 1.4647887	total: 300ms	remaining: 1.08s
217:	learn: 1.4636574	total: 301ms	remaining: 1.08s
218:	learn: 1.4623108	total: 303ms	remaining: 1.08s
219:	learn: 1.4613593	total: 304ms	remaining: 1.08s
220:	learn: 1.4605022	total: 306ms	remaining: 1.08s
221:	learn: 1.4596045	total: 307ms	remaining: 1.07s
222:	learn: 1.4584525	total: 308ms	remaining: 1.07s
223:	learn: 1.4575793	total: 310ms	remaining: 1.07s
224:	learn: 1.4568918	total: 311ms	remaining: 1.07s
225:	learn: 1.4557094	total: 313ms	remaining: 1.07s
226:	learn: 

457:	learn: 1.1667262	total: 625ms	remaining: 739ms
458:	learn: 1.1649945	total: 627ms	remaining: 739ms
459:	learn: 1.1644694	total: 628ms	remaining: 737ms
460:	learn: 1.1639054	total: 629ms	remaining: 736ms
461:	learn: 1.1636675	total: 631ms	remaining: 735ms
462:	learn: 1.1617435	total: 632ms	remaining: 733ms
463:	learn: 1.1617014	total: 633ms	remaining: 732ms
464:	learn: 1.1603868	total: 635ms	remaining: 730ms
465:	learn: 1.1598831	total: 636ms	remaining: 729ms
466:	learn: 1.1593407	total: 638ms	remaining: 728ms
467:	learn: 1.1582470	total: 639ms	remaining: 727ms
468:	learn: 1.1577215	total: 641ms	remaining: 725ms
469:	learn: 1.1572465	total: 642ms	remaining: 724ms
470:	learn: 1.1558946	total: 644ms	remaining: 723ms
471:	learn: 1.1550462	total: 645ms	remaining: 722ms
472:	learn: 1.1547561	total: 646ms	remaining: 720ms
473:	learn: 1.1542745	total: 648ms	remaining: 719ms
474:	learn: 1.1542387	total: 649ms	remaining: 718ms
475:	learn: 1.1539517	total: 651ms	remaining: 716ms
476:	learn: 

708:	learn: 0.9367855	total: 971ms	remaining: 399ms
709:	learn: 0.9359619	total: 973ms	remaining: 398ms
710:	learn: 0.9346107	total: 975ms	remaining: 396ms
711:	learn: 0.9324046	total: 976ms	remaining: 395ms
712:	learn: 0.9302817	total: 978ms	remaining: 394ms
713:	learn: 0.9289800	total: 979ms	remaining: 392ms
714:	learn: 0.9289426	total: 980ms	remaining: 391ms
715:	learn: 0.9275342	total: 982ms	remaining: 389ms
716:	learn: 0.9275099	total: 983ms	remaining: 388ms
717:	learn: 0.9274692	total: 984ms	remaining: 387ms
718:	learn: 0.9266869	total: 986ms	remaining: 385ms
719:	learn: 0.9252667	total: 987ms	remaining: 384ms
720:	learn: 0.9234645	total: 989ms	remaining: 383ms
721:	learn: 0.9227680	total: 990ms	remaining: 381ms
722:	learn: 0.9214580	total: 992ms	remaining: 380ms
723:	learn: 0.9210809	total: 993ms	remaining: 379ms
724:	learn: 0.9207820	total: 994ms	remaining: 377ms
725:	learn: 0.9203160	total: 996ms	remaining: 376ms
726:	learn: 0.9200907	total: 997ms	remaining: 374ms
727:	learn: 

959:	learn: 0.7743484	total: 1.32s	remaining: 54.9ms
960:	learn: 0.7741199	total: 1.32s	remaining: 53.6ms
961:	learn: 0.7740356	total: 1.32s	remaining: 52.2ms
962:	learn: 0.7729850	total: 1.32s	remaining: 50.8ms
963:	learn: 0.7726971	total: 1.32s	remaining: 49.5ms
964:	learn: 0.7722427	total: 1.33s	remaining: 48.1ms
965:	learn: 0.7720044	total: 1.33s	remaining: 46.8ms
966:	learn: 0.7711234	total: 1.33s	remaining: 45.4ms
967:	learn: 0.7710744	total: 1.33s	remaining: 44ms
968:	learn: 0.7708709	total: 1.33s	remaining: 42.7ms
969:	learn: 0.7706078	total: 1.33s	remaining: 41.3ms
970:	learn: 0.7698114	total: 1.34s	remaining: 39.9ms
971:	learn: 0.7689811	total: 1.34s	remaining: 38.5ms
972:	learn: 0.7679876	total: 1.34s	remaining: 37.2ms
973:	learn: 0.7676814	total: 1.34s	remaining: 35.8ms
974:	learn: 0.7674253	total: 1.34s	remaining: 34.4ms
975:	learn: 0.7668542	total: 1.34s	remaining: 33ms
976:	learn: 0.7655077	total: 1.34s	remaining: 31.7ms
977:	learn: 0.7645578	total: 1.35s	remaining: 30.3

---

## Hyperparameter Optimization

### LGB

In [271]:
# def objective(trial):
#     lgb_param = {'n_estimators': trial.suggest_int('n_estimators', 200, 2000),
#            'max_depth': trial.suggest_int('max_depth', 3, 30),
#            #'min_samples_split': trial.suggest_int('min_samples_split', 5, 200),
#            #'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
#            'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 1),
#            'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.1, 1),
#            #'num_leaves' : trial.suggest_uniform('num_leaves', 5, 1000),
#            'reg_alpha': trial.suggest_uniform('reg_alpha', 0.1, 200),
#            'reg_lambda': trial.suggest_uniform('reg_lambda', 0.1, 200)
#           }

#     skf = KFold(n_splits = 10, shuffle=True, random_state=0) 
#     for fold, (tr_ind, val_ind) in enumerate(skf.split(train[features], train[target])):
#         X_train, X_val = train[features].loc[tr_ind], train[features].loc[val_ind]
#         y_train, y_val = train[target][tr_ind], train[target][val_ind]
    
#         model = LGBMRegressor(**lgb_param,random_state=0, n_jobs=-1) 
#         model.fit(X_train, y_train)
#         val_preds = model.predict(X_val) 
#         error = np.sqrt(mean_squared_error(y_val, val_preds))

#         return error
  

    
# study = optuna.create_study(direction="minimize") 
# study.optimize(objective, n_trials=2000)

In [272]:
lgb_params = {'n_estimators': 1862, 'max_depth': 6,
        'learning_rate': 0.09656696890860658, 
        'colsample_bytree': 0.21769935964088244, 
        'reg_alpha': 7.065081242738941, 
        'reg_lambda': 52.701456966072314}

### XGB

In [273]:
# def objective(trial):
#     xgb_param = {'max_depth' : trial.suggest_int('max_depth',2,30),
#     'n_estimators' : trial.suggest_int('n_estimators',1,1000),
#     'learning_rate' : trial.suggest_uniform('learning_rate',0.1,1),
#     'colsample_bytree' : trial.suggest_uniform('colsample_bytree',0.1,0.9),
#     'reg_alpha' : trial.suggest_int('reg_alpha',1,10),
#     'reg_lambda' : trial.suggest_int('reg_lambda',1,10)
#           }

#     skf = KFold(n_splits = 10, shuffle=True, random_state=0) 
#     for fold, (tr_ind, val_ind) in enumerate(skf.split(train[features], train[target])):
#         X_train, X_val = train[features].loc[tr_ind], train[features].loc[val_ind]
#         y_train, y_val = train[target][tr_ind], train[target][val_ind]
    
#         model = XGBRegressor(**xgb_param,random_state=0, n_jobs=-1) 
#         model.fit(X_train, y_train)
#         val_preds = model.predict(X_val) 
#         error = np.sqrt(mean_squared_error(y_val, val_preds))

#         return error
  

    
# study = optuna.create_study(direction="minimize") 
# study.optimize(objective, n_trials=2000)

In [274]:
xgb_params = {'max_depth': 3, 'n_estimators': 958, 
              'learning_rate': 0.29612989262987544, 
              'colsample_bytree': 0.44933960930064265, 
              'reg_alpha': 2, 'reg_lambda': 3}

### Cat

In [275]:
# def objective(trial):
#     cat_param = {'max_depth' : trial.suggest_int("max_depth", 1, 16),
#     'n_estimators' : trial.suggest_int("n_estimators", 200, 1500),
#     'learning_rate' : trial.suggest_uniform('learning_rate', 0.1, 1),
#     'rsm' : trial.suggest_uniform('rsm', 0.1, 0.99),
#     #num_leaves : trial.suggest_int("num_leaves", 2, 5000),
#     #min_child_samples : trial.suggest_int('min_child_samples', 3, 200),
#     'reg_lambda' : trial.suggest_int("reg_lambda", 1, 1000)
#     }

#     skf = KFold(n_splits = 10, shuffle=True, random_state=0) 
#     for fold, (tr_ind, val_ind) in enumerate(skf.split(train[features], train[target])):
#         X_train, X_val = train[features].loc[tr_ind], train[features].loc[val_ind]
#         y_train, y_val = train[target][tr_ind], train[target][val_ind]
    
#         model = CatBoostRegressor(**cat_param,random_state=0) 
#         model.fit(X_train, y_train)
#         val_preds = model.predict(X_val) 
#         error = np.sqrt(mean_squared_error(y_val, val_preds))

#         return error

    
    
# study = optuna.create_study(direction="minimize") 
# study.optimize(objective, n_trials=100)

In [276]:
cat_params = {'max_depth': 14, 'n_estimators': 928, 
              'learning_rate': 0.5252736578880802, 
              'rsm': 0.19104453122990106, 'reg_lambda': 213}

### RandomForest

In [277]:
# def objective(trial):
#     rf_param = {'max_depth':trial.suggest_int("max_depth", 1, 20),
#     'n_estimators' : trial.suggest_int('n_estimators',2, 2000),
#     'min_samples_split' : trial.suggest_int('min_samples_split',2, 200),
#     'min_samples_leaf' : trial.suggest_int('min_samples_leaf',2, 100)
#     }

#     skf = KFold(n_splits = 10, shuffle=True, random_state=0) 
#     for fold, (tr_ind, val_ind) in enumerate(skf.split(train[features], train[target])):
#         X_train, X_val = train[features].loc[tr_ind], train[features].loc[val_ind]
#         y_train, y_val = train[target][tr_ind], train[target][val_ind]
    
#         model = RandomForestRegressor(**rf_param,random_state=0, n_jobs=-1) 
#         model.fit(X_train, y_train)
#         val_preds = model.predict(X_val) 
#         error = np.sqrt(mean_squared_error(y_val, val_preds))

#         return error


# study = optuna.create_study(direction="minimize") 
# study.optimize(objective, n_trials=3000)

In [278]:
rf_params = {'max_depth': 20, 'n_estimators': 6, 'min_samples_split': 2, 'min_samples_leaf': 4}

---

## Submission

In [288]:
sample[target] = new_rf_preds

In [289]:
sample.to_csv(r'D:\Data Science\Tea Story - Hackathon\knn_basic_rf_preds.csv', index = False)

In [290]:
sample
# 118.411840
# 119.093508

,Average
0,119.271778
1,118.528266
2,120.367419
3,121.179426
4,121.244776
5,117.980378
6,119.366476
7,119.565934
8,121.238424
9,118.535766
